In [31]:
import pandas as pd
from sqlalchemy import create_engine, text
from yaml_reader import ConfigLoader

config = ConfigLoader("../configs/config.yaml")
engine = create_engine(
    f"postgresql+psycopg2://{config.get("db.user")}:{config.get("db.password")}@"
    f"{config.get("db.host")}:{config.get("db.port")}/{config.get("db.dbname")}"
)

with engine.connect() as conn:
    result = conn.execute(text("SELECT * from audio_dialog LIMIT 1")).fetchone()
    res = dict(result._asdict()) if result else None
res

pd.set_option('display.max_columns', 500)

In [79]:
df = pd.read_sql(f"""
SELECT
    audio_dialog_fk_id,
    max(t1.has_greeting::int) has_greeting,
    max(t1.has_name::int) has_name,
    array_agg(parasite_words) parasite_words,
    max(t1.has_stopwords::int) has_stopwords,
    max(t1.has_swear_words::int) has_swear_words
-- t2.audio_dialog_fk_id, t1.*, speaker_id
from dialog_criterias t1
left join dialog_rows t2 on t1.dialog_row_fk_id = t2.id
-- left join audio_dialog t3 on t2.audio_dialog_fk_id = t3.id
where speaker_id = 'SALES'
group by 1
""", engine)
df

,audio_dialog_fk_id,has_greeting,has_name,parasite_words,has_stopwords,has_swear_words
0,00cb5b5f-0683-46fa-a412-1ff70e631f94,1,1,"[, , , , , да, да, то есть, , , , , , , , да, ...",1,0
1,0156ca6f-4427-47a6-b6b8-0b1d6f3a1e41,1,1,"[вот, это, , ну, , да, понимаете, да, да, , да...",1,0
2,02d93833-a698-46d9-9f1f-04fae9e465f4,1,1,"[, , да, , , , да, , да, да, то есть, это, , д...",1,0
3,03ee8007-b940-4fe7-9a68-553ce54cbfbe,1,1,"[да, это, , , да, , , да, , ага, да, , , , то ...",1,0
4,03f93284-90a2-4ca9-86be-460289b5c76b,1,1,"[, , , да, , да, , , , , , , , , вот, да, ]",1,0
...,...,...,...,...,...,...
161,fdbd9d5b-cfc5-454c-902d-7896cea7e3fe,0,0,"[это, , ]",0,0
162,fdcba88b-b428-4ae6-861c-f6820ac214af,1,1,"[, , , , , , , , , , , , , , , , да, это, , , ...",1,0
163,feba1f2b-6714-4421-b072-e8a35e355004,1,1,"[, , да, то есть, , , , , ]",1,0
164,fefa337e-f28f-4650-9075-474187d8ae08,1,0,"[получается, , , , , , , , , вот, ну, то есть,...",1,0


In [80]:
df.groupby('has_stopwords').agg({'audio_dialog_fk_id': 'nunique'})
# df[df['has_swear_words'] == 1]

,audio_dialog_fk_id
has_stopwords,
0,13
1,153


In [83]:
df = pd.read_sql(f"""
SELECT file_name, row_text, swear_words, stop_words, t1.*
-- t2.audio_dialog_fk_id, t1.*, speaker_id
from dialog_rows t2
left join dialog_criterias t1 on t1.dialog_row_fk_id = t2.id
left join audio_dialog t3 on t2.audio_dialog_fk_id = t3.id
where audio_dialog_fk_id = 'fb18a776-3b4b-44ec-962a-7cdfea0bb104'
and speaker_id = 'SALES'
-- group by 1
and has_stopwords = True
order by row_num
""", engine)
df

,file_name,row_text,swear_words,stop_words,dialog_criteria_id,dialog_row_fk_id,has_greeting,greeting_phrase,has_name,found_name,has_farewell,farewell_phrase,interjections,parasite_words,abbreviations,slang,inappropriate_phrases,has_diminutives,diminutives,has_stopwords,stop_words,has_swear_words,swear_words
0,in-07010101-83412933199-20250402-111618-174358...,"Добрый день, Вячеслав. Какие могут обращаться?",,не могу,55cb2f28-3537-4b06-a7eb-6e52faf973d4,91120ec0-0ace-489c-8cac-3c09bf93c78e,True,добрый день,False,None,False,None,,,,,,False,,True,не могу,False,
1,in-07010101-83412933199-20250402-111618-174358...,"Я вам подскажу, что у вас не Михаил, уточняйте...",,нет информации,9dc32fe3-d105-4b9c-a8d3-38f367c05da8,b7cad435-6de5-4682-9e4a-b7fa85acc1ca,False,None,False,None,False,None,,,,,,False,,True,нет информации,False,
2,in-07010101-83412933199-20250402-111618-174358...,В сезонном монтаже они могут работать и до веч...,,не могу,c52418e0-f402-42ea-8195-32389f114eed,0198f595-ebfd-445f-9ac8-9197434974ad,False,None,False,None,False,None,,,,,,False,,True,не могу,False,
3,in-07010101-83412933199-20250402-111618-174358...,"Турвейный комплект. Так, две в наличии только...",,у нас нет,bd07e2cb-04b8-409f-9fc8-346dc9af5aed,5e38dc56-24d3-409b-aa05-5a4ec00984ee,False,None,False,None,False,None,,,,,,False,,True,у нас нет,False,
4,in-07010101-83412933199-20250402-111618-174358...,"Да, я вам сейчас озвучу, давайте я вам по айко...",хуйно,"у нас нет, я же говорил",a6a4709e-86a7-4edb-a556-f474811cb207,91e80f92-f769-48df-96b6-636ac829e713,False,None,True,Нордман,False,None,"а, это","да, то есть, это",,,,False,,True,"у нас нет, я же говорил",True,хуйно
5,in-07010101-83412933199-20250402-111618-174358...,"На 7 апреля, да? Да. Я вам озвучу информацию. ...",,вы должны,11467d3f-076a-4ebd-b2b3-608980ec1529,87944369-4a90-4ef3-8a22-88fec8f721a3,False,None,False,None,False,None,это,"да, то есть, это",,,,False,,True,вы должны,False,
6,in-07010101-83412933199-20250402-111618-174358...,"И как раз с пиццом уже подкашьте. Ну, нормы вп...",,не могу,b6d3b707-0d59-4f8e-9a42-3efc7893f111,fd6e8a67-cdcd-49d0-afab-1c89056a388b,False,None,False,None,False,None,,ну,,,,False,,True,не могу,False,


In [88]:
df['row_text'][6]

'И как раз с пиццом уже подкашьте. Ну, нормы вполне могут быть еще. Хорошо. '